### Regularization Tutorial

Welcome to the ART regularization tutorial. At this point we assume that you are quite confident about your model and now your goal is to achieve reasonable accuracy. We will start from the `Overfit` stage. If you are not familiar with ART consider checking other [tutorials](https://github.com/SebChw/Actually-Robust-Training#tutorials).


We wrote `dataset.py`, `modifiers.py` and `models.base_model.py` for you. If you are interested please check them out! Everything is described inside them.

Let's start from the project definition.

In [ ]:
# Install required packages
!pip install kornia

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#Set seed for reproducibility
from lightning import seed_everything
seed_everything(23)

In [ ]:
import torch.nn as nn
from dataset import FruitsDataModule, N_CLASSES
from models.base_model import FoodClassifier
from torchmetrics import Accuracy
from art.project import ArtProject

data_module = FruitsDataModule() # Datamodule on which we try to achieve good performance
model = FoodClassifier # Model class with which we want to achieve it
project = ArtProject("regularize", data_module) # Create a project with a name and a datamodule
#Register metrics to be tracked
loss, accuracy = nn.CrossEntropyLoss(), Accuracy(task="multiclass", num_classes=N_CLASSES)
project.register_metrics([loss, accuracy])

At first, we define a Model Callback that will save models with best validation scores

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint
from art.metrics import build_metric_name
from art.utils.enums import TrainingStage

checkpoint = ModelCheckpoint(monitor=build_metric_name(accuracy, TrainingStage.VALIDATION.value), mode="max")

Here you can observe quite important philosophy of ART we try to omit MAGIC values everything is defined programmaticaly.

Next, lets define a check to validate our regularization progress. Our client will be satisfied with 75% accuracy. 

In [ ]:
from art.checks import CheckScoreGreaterThan

WANTED_SCORE = 0.75
acc_check = CheckScoreGreaterThan(accuracy, WANTED_SCORE)

Finnaly Lets assume that we will be always doing 10 epochs of training.

In [ ]:
TRAINER_KWARGS = {"callbacks": [checkpoint], "max_epochs": 10}

We start from no regularization. Maybe it's not needed?

In [ ]:
from art.steps import Regularize

project.add_step(Regularize(model), checks = [acc_check])
project.run_all(trainer_kwargs=TRAINER_KWARGS)

Unfortunatelly it is needed. Generally we should try to introduce regularizations from these that are most certain to improve out score:

1. Get more training data -  hard, although it's the only guaranteed way to monotonically improve the performance of a well-configured neural network almost indefinitely.
2. Data augmentations
3. Generating fake data
4. Pretraining
5. Decreasing the network complexity
6. Decreasing the batch size, weight decay, dropout etc.
7. Early Stopping


In [ ]:
from modifiers import AddMoreDataModifier

project.add_step(Regularize(model, datamodule_modifiers=[AddMoreDataModifier()]), checks = [acc_check])
project.run_all(trainer_kwargs=TRAINER_KWARGS)

Observe that the previous Regularize step was not executed again.

So the ART idea for regularization is to achieved by `modifiers`. These are functions that modify something in the model or dataloader to achieve different trainings. By utilizing modifiers we can track them with `git`. Moreover we are not expanding our model and data classes.

As you can see adding more data as always helps. But we are still not there.

Now we add 2 runs: one with little and second with many augmentations.

In [ ]:
from modifiers import SetLittleTransformsModifier, SetManyTransformsModifier

project.add_step(Regularize(model, datamodule_modifiers=[AddMoreDataModifier()],
                            model_modifiers=[SetLittleTransformsModifier()]), checks = [acc_check])
project.add_step(Regularize(model, datamodule_modifiers=[AddMoreDataModifier()],
                            model_modifiers=[SetManyTransformsModifier()]), checks = [acc_check])

project.run_all(trainer_kwargs=TRAINER_KWARGS)

Eventually there is a third option - we can just pass kwargs to our model.

In [ ]:
project.add_step(Regularize(model, datamodule_modifiers=[AddMoreDataModifier()],
                            model_modifiers=[SetManyTransformsModifier()], 
                            model_kwargs={"weight_decay": 0.5}), checks = [acc_check])

project.run_all(trainer_kwargs=TRAINER_KWARGS)

As you can see when we suceeded with regularization next trials won't be run.

Finally, you can now visualize and explore all runs using art-dashboard - `python -m art.cli run-dashboard`

To guide you step by step we were introducing new regularization concept one by one and use jupyter notebook. However, you can easily write exactly the same code using script. Check `run.py` for this purpose.